In [22]:
from keras.preprocessing import image
from keras.models import load_model
import cv2
import numpy as np
import os

In [23]:
Rev_class_map={
    0:'paper',
    1:'rock',
    2:'scissors'
}
def mapper(val):
    return Rev_class_map[val]

In [24]:
# here move1 is user,move2 is computer
def calculate_winner(move1,move2):
    if move1==move2:
        return "Tie"
    if move1=="rock":
        if move2=="scissors":
            return "User"
        if move2=="paper":
            return "Computer"
    if move1=="scissors":
        if move2=="rock":
            return "Computer"
        if move2=="paper":
            return "User"
    if move1=="paper":
        if move2=="rock":
            return "user"
        if move2=="scissors":
            return "Computer"    



In [25]:
#load model
model=load_model("rps.h2")
   

In [ ]:
cap=cv2.VideoCapture(0)

def increase_screen():
    cap.set(3,1366)
    cap.set(4,720)

increase_screen()
prev_move=None
while True:
    ret,frame=cap.read()
    if ret==False:
        continue
    #rectangle for user to play
    cv2.rectangle(frame,(100,100),(500,500),(0,0,255),2)
    #rectangle for computer to play
    cv2.rectangle(frame,(800,100),(1200,500),(255,0,0),2)
    #extract the region of image within the user rectangle
    roi=frame[100:500,100:500]
    img=cv2.cvtColor(roi,cv2.COLOR_BGR2RGB)
    img=cv2.resize(img,(150,150))
    
    img=np.array(img)
    img = np.expand_dims(img, axis=0)
    pred=model.predict(img)
    move_code=np.argmax(pred)
    user_move_name=mapper(move_code)
    #predict the winner
    if prev_move!=user_move_name:
        computer_move_name=np.random.choice(['rock','paper','scissors'])
        winner=calculate_winner(user_move_name,computer_move_name)
    prev_move=user_move_name
    #display the info
    font=cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame,"Your Move : "+user_move_name,(50,50),font,1.2,(0,0,255),2,cv2.LINE_AA)
    cv2.putText(frame,"Computer's Move : "+computer_move_name,(750,50),font,1.2,(255,0,0),2,cv2.LINE_AA)
    cv2.putText(frame,"Winner : "+winner,(400,600),font,2,(0,255,0),4,cv2.LINE_AA)
    
    icon=cv2.imread("images/{}.png".format(computer_move_name))
    icon=cv2.resize(icon,(400,400))
    frame[100:500,800:1200]=icon
    cv2.imshow("Rock Paper Scissor",frame)    
        
    key_pressed=cv2.waitKey(10) 
    if key_pressed==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()